In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls /content/drive/My\ Drive/archive1

training.1600000.processed.noemoticon.csv


In [3]:
import os
cols = ["target","id","date","flag","user","text"]
path_file = "/content/drive/My Drive/archive1/training.1600000.processed.noemoticon.csv"
print(os.path.exists(path_file))

True


In [4]:
import pandas as pd
data = pd.read_csv(
    path_file,
    engine= "python",
    on_bad_lines = "skip",
    encoding='latin1',
    names = cols
)
data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [5]:
data.shape

(1600000, 6)

In [6]:
data = data[["target","text"]]
data["target"] = data["target"].replace({0:0,2:1,4:2})

In [7]:
data["target"].value_counts()

,count
target,
0,800000
2,800000


In [8]:
from sklearn.model_selection import train_test_split
small_data,_ = train_test_split(
    data,
    train_size=0.005,
    stratify= data["target"],
    random_state=42
)

In [9]:
data = small_data.reset_index(drop=True)
data.head()

,target,text
0,2,hey@bellarocker hi hally girl you're so great ...
1,2,Hit bench for the first time in 6 months. 220...
2,0,Just on my way to start nightshift
3,0,"Nope, it is my laptop power socket which is br..."
4,2,"@TradeSecretLaw Scott, graduation congratulati..."


In [10]:
data["target"].value_counts()

,count
target,
2,4000
0,4000


In [11]:
import re
import string
from nltk.corpus import stopwords
import nltk

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

def clean_text(text):
  text = text.lower()
  text = re.sub(r"http\S+|www\S+","",text)
  text = text.translate(str.maketrans("","",string.punctuation))
  text = re.sub(r"\d+","",text)
  words = text.split()
  words = [w for w in words if w not in stop_words]
  return "".join(words)


data["clean_text"] = data["text"].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
from sklearn.model_selection import train_test_split

X = data["clean_text"]
y = data["target"]

x_train,x_test,y_train,y_test = train_test_split(
    X,y,
    test_size=0.30,
    stratify = y,
    random_state = 42
)

In [40]:
import tensorflow as tf
from tensorflow.keras import layers,models

In [41]:
def built_fc_model(input_dim):
  model = models.Sequential([
      layers.Dense(32,activation="relu",input_shape=(input_dim,)),
      layers.Dense(16,activation="relu"),
      layers.Dense(1,activation="sigmoid")

  ])
  model.compile(
      optimizer="adam",
      loss="binary_crossentropy",
      metrics = ["accuracy"])
  return model



In [42]:
vectorizer_1gm = layers.TextVectorization(
    max_tokens = 10000,
    output_mode = "multi_hot",
    ngrams = 1

)

In [43]:
vectorizer_1gm.adapt(x_train)


In [44]:
x_train_vec = vectorizer_1gm(x_train)
x_test_vec = vectorizer_1gm(x_test)

In [45]:
x_train_vec.shape[1]

5588

In [46]:
model1 = built_fc_model(x_train_vec.shape[1])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [47]:
model1.fit(x_train_vec,y_train,epochs=5,validation_split=0.2)

Epoch 1/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.0010 - loss: 0.5129 - val_accuracy: 0.0000e+00 - val_loss: 0.1061
Epoch 2/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0000e+00 - loss: -0.1317 - val_accuracy: 0.0000e+00 - val_loss: 0.1048
Epoch 3/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0000e+00 - loss: -0.8419 - val_accuracy: 0.0000e+00 - val_loss: 0.1810
Epoch 4/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.0000e+00 - loss: -3.1257 - val_accuracy: 0.0000e+00 - val_loss: 0.3250
Epoch 5/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0000e+00 - loss: -10.1737 - val_accuracy: 0.0000e+00 - val_loss: 0.6102


In [48]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
def evaluate(model,x_test,y_test):
  y_pred = model.predict(x_test)
  y_pred = y_pred.argmax(axis=1)

  acc = accuracy_score(y_test,y_test)
  p,r,f,_ = precision_recall_fscore_support(
      y_test,
      y_pred,
      average = "weighted"
  )
  return acc,p,r,f



x_test_vec = vectorizer_1gm(x_test)
acc,p,r,f = evaluate(model1,x_test_vec,y_test)

print("accuracy:",acc)
print("precision:",p)
print("recall:",r)
print("f1 score:",f)


75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
accuracy: 1.0
precision: 0.25
recall: 0.5
f1 score: 0.3333333333333333


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [49]:
vectorizer_2gm = layers.TextVectorization(
    max_tokens = 10000,
    output_mode = "multi_hot",
    ngrams = 2
)

vectorizer_2gm.adapt(x_train)

In [50]:
x_train_vec = vectorizer_2gm(x_train)
x_test_vec = vectorizer_2gm(x_test)

In [51]:
model_2 = built_fc_model(x_train_vec.shape[1])
model_2.fit(x_train_vec,y_train,validation_split=0.2,epochs=5)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.0107 - loss: 0.5255 - val_accuracy: 0.0000e+00 - val_loss: 0.1151
Epoch 2/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0000e+00 - loss: -0.1064 - val_accuracy: 0.0000e+00 - val_loss: 0.0989
Epoch 3/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0000e+00 - loss: -0.7631 - val_accuracy: 0.0000e+00 - val_loss: 0.1603
Epoch 4/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0000e+00 - loss: -2.9260 - val_accuracy: 8.9286e-04 - val_loss: 0.2285
Epoch 5/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 7.6352e-04 - loss: -10.2083 - val_accuracy: 0.0018 - val_loss: 0.3785


In [52]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
def evaluate(model,x_test,y_test):
  y_pred = model_2.predict(x_test)
  y_pred = y_pred.argmax(axis=1)
  acc = accuracy_score(y_test,y_pred)
  p,r,f,_ = precision_recall_fscore_support(
      y_test,
      y_pred,
      average = "weighted"
  )
  return acc,p,r,f



x_test_vec = vectorizer_2gm(x_test)
acc, p, r, f = evaluate(model_2,x_test_vec,y_test)

print("accuracy:",acc)
print("precision:",p)
print("recall:",r)
print("f1 score:",f)

75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
accuracy: 0.5
precision: 0.25
recall: 0.5
f1 score: 0.3333333333333333


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [53]:
vectorizer_tfidf = layers.TextVectorization(
    max_tokens = 10000,
    output_mode = "tf_idf",
    ngrams = 2
)

vectorizer_tfidf.adapt(x_train)

In [54]:
x_train_vec = vectorizer_tfidf(x_train)
x_test_vec = vectorizer_tfidf(x_test)

In [55]:
model_3 = built_fc_model(x_train_vec.shape[1])
model_3.fit(x_train_vec,y_train,validation_split=0.2,epochs=5)

Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


140/140 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.0124 - loss: 0.5303 - val_accuracy: 0.0000e+00 - val_loss: 0.1658
Epoch 2/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0000e+00 - loss: -0.5934 - val_accuracy: 0.0000e+00 - val_loss: 0.1276
Epoch 3/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: -5.9381 - val_accuracy: 8.9286e-04 - val_loss: 0.4776
Epoch 4/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0012 - loss: -30.5806 - val_accuracy: 8.9286e-04 - val_loss: 1.3613
Epoch 5/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0227 - loss: -98.4990 - val_accuracy: 0.0018 - val_loss: 2.8627


In [56]:
def evaluate(model,x_test,y_test):
  y_pred = model_3.predict(x_test)
  y_pred = y_pred.argmax(axis=1)

  acc = accuracy_score(y_test,y_pred)
  p,r,f,_ = precision_recall_fscore_support(
      y_test,
      y_pred,
      average = "weighted"
  )
  return acc,p,r,f



x_test_vec_tfidf = vectorizer_tfidf(x_test)
acc,p,r,f = evaluate(model_3,x_test_vec_tfidf,y_test)
print("accuracy:",acc)
print("precision:",p)
print("recall:",r)
print("f1 score:",f)

75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
accuracy: 0.5
precision: 0.25
recall: 0.5
f1 score: 0.3333333333333333


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [57]:
vectorizer_seq = layers.TextVectorization(
    max_tokens =10000,
    output_mode = "int",
    output_sequence_length = 140
)

vectorizer_seq.adapt(x_train)


In [58]:
x_train_seq =  vectorizer_seq(x_train)
x_test_seq = vectorizer_seq(x_test)

In [59]:
model_lstm = models.Sequential([
    layers.Embedding(10000,128),
    layers.LSTM(32),
    layers.Dense(1,activation="sigmoid")
])
model_lstm.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model_lstm.fit(x_train_seq, y_train, validation_split=0.2, epochs=5)

Epoch 1/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.0233 - loss: 0.1135 - val_accuracy: 0.0000e+00 - val_loss: 0.1051
Epoch 2/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.0000e+00 - loss: -0.0258 - val_accuracy: 0.0000e+00 - val_loss: 0.1155
Epoch 3/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0000e+00 - loss: -0.0809 - val_accuracy: 0.0000e+00 - val_loss: 0.1148
Epoch 4/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.0000e+00 - loss: 0.0825 - val_accuracy: 0.0000e+00 - val_loss: 0.1273
Epoch 5/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.0000e+00 - loss: 0.0276 - val_accuracy: 0.0000e+00 - val_loss: 0.1296


In [60]:
def evaluate(model,x_test,y_test):
  y_pred = model_lstm.predict(x_test)
  y_pred = y_pred.argmax(axis=1)

  acc = accuracy_score(y_test,y_pred)
  p,r,f,_ = precision_recall_fscore_support(
      y_test,
      y_pred,
      average = "weighted"
  )
  return acc,p,r,f



x_test_vec_tfidf = vectorizer_seq(x_test)
acc,p,r,f = evaluate(model_lstm,x_test_vec_tfidf,y_test)
print("accuracy:",acc)
print("precision:",p)
print("recall:",r)
print("f1 score:",f)

75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
accuracy: 0.5
precision: 0.25
recall: 0.5
f1 score: 0.3333333333333333


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [69]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip -q glove.6B.zip
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip
!unzip -q glove.twitter.27B.zip

--2026-02-27 02:50:28--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2026-02-27 02:50:28--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2026-02-27 02:50:29--  https://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [ap

In [62]:
import numpy as np

embedding_dim = 100
glove_path = "glove.6B.100d.txt"

embeddings_index = {}

with open(glove_path, encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype="float32")
        embeddings_index[word] = vector

print("Loaded word vectors:", len(embeddings_index))

Loaded word vectors: 400000


In [63]:
vectorizer_seq1 = layers.TextVectorization(
    max_tokens=10000,
    output_mode="int",
    output_sequence_length=140
)

vectorizer_seq1.adapt(x_train)

x_train_seq = vectorizer_seq1(x_train)
x_test_seq  = vectorizer_seq1(x_test)

In [64]:
vocab = vectorizer_seq1.get_vocabulary()
word_index = dict(zip(vocab, range(len(vocab))))

num_tokens = len(vocab)

embedding_matrix = np.zeros((num_tokens, embedding_dim))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [65]:
model_lstm1 = models.Sequential([
    layers.Embedding(
        input_dim=num_tokens,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        trainable=False   # important for GloVe
    ),
    layers.LSTM(32),
    layers.Dense(1, activation="sigmoid")
])

model_lstm1.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model_lstm1.fit(
    x_train_seq,
    y_train,
    validation_split=0.2,
    epochs=5
)

Epoch 1/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.0172 - loss: 0.2929 - val_accuracy: 0.0000e+00 - val_loss: 0.0987
Epoch 2/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.0000e+00 - loss: -0.0939 - val_accuracy: 0.0000e+00 - val_loss: 0.1044
Epoch 3/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0000e+00 - loss: -0.0709 - val_accuracy: 0.0000e+00 - val_loss: 0.1120
Epoch 4/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.0000e+00 - loss: 0.1016 - val_accuracy: 0.0000e+00 - val_loss: 0.1138
Epoch 5/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.0000e+00 - loss: -0.0621 - val_accuracy: 0.0000e+00 - val_loss: 0.1243


In [66]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def evaluate(model, x_test, y_test):
    y_pred = model_lstm1.predict(x_test)
    y_pred = y_pred.argmax(axis=1)

    acc = accuracy_score(y_test, y_pred)
    p, r, f, _ = precision_recall_fscore_support(
        y_test, y_pred, average="weighted"
    )
    return acc, p, r, f

acc, p, r, f = evaluate(model_lstm1, x_test_seq, y_test)

print("accuracy:", acc)
print("precision:", p)
print("recall:", r)
print("f1 score:", f)


75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
accuracy: 0.5
precision: 0.25
recall: 0.5
f1 score: 0.3333333333333333


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [70]:
import numpy as np

embedding_dim = 100
glove_path = "glove.twitter.27B.100d.txt"

embeddings_index = {}

with open(glove_path, encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype="float32")
        embeddings_index[word] = vector

print("Loaded word vectors:", len(embeddings_index))

Loaded word vectors: 1193514


In [71]:
vectorizer_seq1 = layers.TextVectorization(
    max_tokens=10000,
    output_mode="int",
    output_sequence_length=140
)

vectorizer_seq1.adapt(x_train)

x_train_seq = vectorizer_seq1(x_train)
x_test_seq  = vectorizer_seq1(x_test)

In [72]:
vocab = vectorizer_seq1.get_vocabulary()
word_index = dict(zip(vocab, range(len(vocab))))

num_tokens = len(vocab)

embedding_matrix = np.zeros((num_tokens, embedding_dim))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [73]:
model_lstm1 = models.Sequential([
    layers.Embedding(
        input_dim=num_tokens,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        trainable=False   # important for GloVe
    ),
    layers.LSTM(32),
    layers.Dense(1, activation="sigmoid")
])

model_lstm1.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model_lstm1.fit(
    x_train_seq,
    y_train,
    validation_split=0.2,
    epochs=5
)

Epoch 1/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.0208 - loss: 0.2968 - val_accuracy: 0.0000e+00 - val_loss: 0.0934
Epoch 2/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.0000e+00 - loss: 0.0973 - val_accuracy: 0.0000e+00 - val_loss: 0.1001
Epoch 3/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.0000e+00 - loss: 0.0806 - val_accuracy: 0.0000e+00 - val_loss: 0.1066
Epoch 4/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.0000e+00 - loss: 0.0648 - val_accuracy: 0.0000e+00 - val_loss: 0.1196
Epoch 5/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.0000e+00 - loss: 0.1045 - val_accuracy: 0.0000e+00 - val_loss: 0.1235


In [74]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def evaluate(model, x_test, y_test):
    y_pred = model_lstm1.predict(x_test)
    y_pred = y_pred.argmax(axis=1)

    acc = accuracy_score(y_test, y_pred)
    p, r, f, _ = precision_recall_fscore_support(
        y_test, y_pred, average="weighted"
    )
    return acc, p, r, f

acc, p, r, f = evaluate(model_lstm1, x_test_seq, y_test)

print("accuracy:", acc)
print("precision:", p)
print("recall:", r)
print("f1 score:", f)


75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
accuracy: 0.5
precision: 0.25
recall: 0.5
f1 score: 0.3333333333333333


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [76]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
!gunzip cc.en.300.vec.gz

--2026-02-27 03:16:21--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 65.8.76.47, 65.8.76.89, 65.8.76.35, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|65.8.76.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1325960915 (1.2G) [binary/octet-stream]
Saving to: ‘cc.en.300.vec.gz’

cc.en.300.vec.gz    100%[===================>]   1.23G   200MB/s    in 12s     

2026-02-27 03:16:32 (109 MB/s) - ‘cc.en.300.vec.gz’ saved [1325960915/1325960915]



In [77]:
import numpy as np

embedding_dim = 300
fasttext_path = "cc.en.300.vec"

embeddings_index = {}

with open(fasttext_path, encoding="utf8", newline="\n", errors="ignore") as f:
    next(f)   # skip header
    for line in f:
        values = line.rstrip().split(" ")
        word = values[0]
        vector = np.asarray(values[1:], dtype="float32")
        embeddings_index[word] = vector

print("Loaded FastText vectors:", len(embeddings_index))

Loaded FastText vectors: 2000000


In [78]:
vocab = vectorizer_seq.get_vocabulary()
word_index = dict(zip(vocab, range(len(vocab))))

num_tokens = len(vocab)
embedding_matrix = np.zeros((num_tokens, embedding_dim))

hits, misses = 0, 0

for word, i in word_index.items():
    vector = embeddings_index.get(word)
    if vector is not None:
        embedding_matrix[i] = vector
        hits += 1
    else:
        misses += 1

print("hits:", hits)
print("misses:", misses)

hits: 64
misses: 5525


In [79]:
model_lstm = models.Sequential([
    layers.Embedding(
        input_dim=num_tokens,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        trainable=True   # IMPORTANT: fine-tune FastText
    ),
    layers.LSTM(32),
    layers.Dense(1, activation="sigmoid")
])

model_lstm.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model_lstm.fit(
    x_train_seq,
    y_train,
    validation_split=0.2,
    epochs=5
)

Epoch 1/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.0184 - loss: 0.0954 - val_accuracy: 0.0000e+00 - val_loss: 0.0999
Epoch 2/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.0000e+00 - loss: 0.0492 - val_accuracy: 0.0000e+00 - val_loss: 0.1136
Epoch 3/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.0000e+00 - loss: 0.1166 - val_accuracy: 0.0000e+00 - val_loss: 0.1203
Epoch 4/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.0000e+00 - loss: -0.1319 - val_accuracy: 0.0000e+00 - val_loss: 0.1340
Epoch 5/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.0000e+00 - loss: 0.1445 - val_accuracy: 0.0000e+00 - val_loss: 0.1348


In [80]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def evaluate(model, x_test, y_test):
    y_pred = model.predict(x_test)
    y_pred = y_pred.argmax(axis=1)

    acc = accuracy_score(y_test, y_pred)
    p, r, f, _ = precision_recall_fscore_support(
        y_test, y_pred, average="weighted"
    )
    return acc, p, r, f



acc, p, r, f = evaluate(model_lstm, x_test_seq, y_test)

print("accuracy:", acc)
print("precision:", p)
print("recall:", r)
print("f1 score:", f)

75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
accuracy: 0.5
precision: 0.25
recall: 0.5
f1 score: 0.3333333333333333


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
